# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [25]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt
import re
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage

In [4]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gemini-2.0-flash'
model = ChatGoogleGenerativeAI(
    api_key=api_key,
    model=MODEL,
    temperature=0.7,        # Controls randomness (0 = deterministic)
    top_k=40,               # Optional: limits sampling to top-k tokens
    top_p=0.9               # Optional: nucleus sampling
)

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [15]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [9]:
link_system_prompt = """You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.

You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""


In [10]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.

You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [13]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links_user_prompt(website):
    """
    Create a user prompt listing all links on a website and asking
    which are relevant for a company brochure.
    """
    user_prompt = (
        f"Here is the list of links on the website of {website.url}.\n"
        "Please decide which of these links are relevant for a brochure about the company.\n"
        "Respond in JSON format with the full https URL.\n"
        "Do not include Terms of Service, Privacy Policy, or email links.\n\n"
        "Links (some might be relative):\n"
        f"{chr(10).join(website.links)}"
    )
    return user_prompt


In [14]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com.
Please decide which of these links are relevant for a brochure about the company.
Respond in JSON format with the full https URL.
Do not include Terms of Service, Privacy Policy, or email links.

Links (some might be relative):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [26]:
def get_links(url):
    website = Website(url)
    # Create messages
    messages = [
        SystemMessage(content=link_system_prompt),
        HumanMessage(content=get_links_user_prompt(website))
    ]
    # Get response
    response = model.invoke(messages)
    text = response.content.strip()

    # Extract JSON object using regex
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        raise ValueError("No JSON found in the model output")

    json_text = match.group(0)
    return json.loads(json_text)

In [18]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://anthropic.com")
huggingface.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/claude-code',
 'https://www.anthropic.com/max',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/code-modernization',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/solutions/education',
 'https://www.anthropic.com/solutions/financial-services',
 'https://www.anthropic.com/solutions/government',
 'https://www.anthropic.com/customers',
 'https://www.

In [27]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing', 'url': 'https://huggingface.co/pricing'},
  {'type': 'brand', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'join', 'url': 'https://huggingface.co/join'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [41]:
Website('https://anthropic.com').get_contents()

"Webpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nProduct\nClaude\nClaude Code\nPlans\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s Character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCode modernization\nCoding\nCustomer support\nEducation\nFinancial services\nGovernment\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Futures\nClaude model family\nClaude Opus 4.1\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification\nLearn\nLearning resources\nCustomer stories\nEngineering at Anthropic\nAnthropic Academy\nCompany\nAbout\nCareers\nEvents\nEngineering\nB

In [42]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [44]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'jobs page', 'url': 'https://www.anthropic.com/jobs'}, {'type': 'products page', 'url': 'https://www.anthropic.com/claude'}, {'type': 'products page', 'url': 'https://www.anthropic.com/claude-code'}, {'type': 'products page', 'url': 'https://www.anthropic.com/max'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/code-modernization'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'solutions page', 'url': 'htt

In [115]:
system_prompt = (
    "You are an expert marketing copywriter and corporate communications specialist.\n"
    "Your task is to create a concise and professional company brochure in Markdown format. Do NOT use triple backticks around the output. Only generate plain Markdown\n "
    "for stakeholders (investors, partners, and clients).\n\n"
    "You will be given text extracted from multiple pages of a company’s website.\n\n"
    "Instructions:\n"
    "1. Review the text carefully and ignore irrelevant details (menus, disclaimers, UI text).\n"
    "2. Identify the main company messages (mission, vision, values, products, services, careers).\n"
    "3. Summarize and rewrite into clear, persuasive brochure copy.\n"
    "4. Ensure the tone is professional, inspiring, and trustworthy.\n"
    "5. Output should be Markdown-formatted, with headings and sections and strictly do not give in code format with triplebackticks\n\n"
)

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [116]:
def get_brochure_user_prompt(company_name, url, max_length=500000):
    # Intro instructions
    user_prompt = (
        f"You are looking at a company called: {company_name}\n"
        f"Here are the contents of its landing page and other relevant pages.\n"
        f"Use this information to build a short brochure of the company in markdown.\n"
        f"Make sure the result is clear, professional, and concise.\n"
        f"---\n"
    )
    
    # Append website details
    content = get_all_details(url)
    combined_prompt = user_prompt + content

    # Truncate if longer than max_length
    if len(combined_prompt) > max_length:
        combined_prompt = combined_prompt[:max_length]

    return combined_prompt

In [118]:
get_brochure_user_prompt("HuggingFace", "https://anthropic.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/jobs'}, {'type': 'products', 'url': 'https://www.anthropic.com/claude'}, {'type': 'products', 'url': 'https://www.anthropic.com/claude-code'}, {'type': 'products', 'url': 'https://www.anthropic.com/max'}, {'type': 'enterprise', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'pricing', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'API', 'url': 'https://www.anthropic.com/api'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/code-modernization'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/coding'}, {'type': 'solution

'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages.\nUse this information to build a short brochure of the company in markdown.\nMake sure the result is clear, professional, and concise.\n---\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nProduct\nClaude\nClaude Code\nPlans\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s Character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCode modernization\nCoding\nCustomer support\nEducation\nFinancial services\nGovernment\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Futures\nClaude model family\nClaude Opus 4.1\nClaude Sonnet 4\nClaude Haiku 3.5\nResearch\nClaude’s extended thinking\nCommitments\

In [110]:
def create_brochure(company_name, url):
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=get_brochure_user_prompt(company_name, url))
    ]
    
    # Invoke the model
    response = model.invoke(messages)
    
    # LangChain response comes as `AIMessage`
    result = response.content if response else "⚠️ No response generated."
    
    # Show in Markdown if in Jupyter
    return result


In [119]:
result = create_brochure("HuggingFace", "https://anthropic.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/jobs'}, {'type': 'products', 'url': 'https://www.anthropic.com/claude'}, {'type': 'products', 'url': 'https://www.anthropic.com/claude-code'}, {'type': 'products', 'url': 'https://www.anthropic.com/max'}, {'type': 'enterprise', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'customers', 'url': 'https://www.anthropic.com/customers'}, {'type': 'research', 'url': 'https://www.anthropic.com/research'}, {'type': 'news', 'url': 'https://www.anthropic.com/news'}, {'type': 'events', 'url': 'https://www.anthropic.com/events'}, {'type': 'engineering', 'url': 'https://www.anthropic.com/engineering'}, {'type': 'learn', 'url': 'https://www.anthropic.c

In [ ]:
result

In [113]:
display(Markdown(result))

# Anthropic: AI for Humanity's Long-Term Well-Being

## About Anthropic

Anthropic is an AI safety and research company dedicated to building reliable, interpretable, and steerable AI systems that benefit humanity. We believe AI will have a vast impact on the world, and we are committed to developing systems that people can rely on while proactively addressing the opportunities and risks of AI.

## Our Mission

To research, develop, and deploy AI systems that prioritize safety and deliver tangible benefits to businesses, nonprofits, civil society groups, and individuals globally. We strive to show what responsible AI development looks like in practice through our research, policy work, and product design.

## Core Values

*   **Act for the global good:** Maximize positive outcomes for humanity in the long run.
*   **Hold light and shade:** Understand and protect against potential risks while realizing the benefits of AI.
*   **Be good to our users:** Cultivate generosity and kindness in all interactions.
*   **Ignite a race to the top on safety:** Inspire competition in developing the safest and most secure AI systems.
*   **Do the simple thing that works:** Take an empirical approach and prioritize impact over sophistication.
*   **Be helpful, honest, and harmless:** Communicate directly and thoughtfully, assuming good intentions.
*   **Put the mission first:** Ensure a shared purpose and swift action through mission-driven decision-making.

## Products and Solutions

Anthropic's flagship product, **Claude**, is an AI assistant designed to be helpful, harmless, and honest. Claude is available through various plans to meet diverse needs:

*   **Claude:** Your thinking partner, connecting to your world and expanding what you can do individually or with your team.
*   **Claude Code:** Empower your startup or team with AI-powered development and collaboration.
*   **Max Plan**: For individuals needing more usage.
*   **Team Plan:** Designed for easy collaboration and better outcomes.
*   **Enterprise Plan:** Securely connect Claude to your company knowledge and empower every team with trusted AI.
*   **Claude Gov Models**: Designed for classified environments for national security missions.

**Key Solutions:**

*   **AI Agents:** Build powerful, collaborative, and safe AI agents with Claude.
*   **Code Modernization:** Accelerate the modernization of legacy codebases with Claude Code.
*   **Coding:** Write beautiful code and ship powerful products with Claude.
*   **Customer Support:** Build AI support agents with a more human touch.
*   **Education:** Navigate AI in education with tools designed for true learning and academic integrity.
*   **Financial Services:** Transform financial services with a unified intelligence layer for institutional finance.
*   **Government:** Deploy responsible AI that meets government needs with rigorous security controls.

## Research and Commitments

Anthropic is committed to:

*   **Frontier AI Research:** Conducting research across modalities to create safer, steerable, and more reliable models.
*   **Transparency:** Sharing insights and learnings with the world.
*   **Responsible Scaling Policy:** Ensuring the safe and ethical development and deployment of AI.
*   **Security and Compliance:** Maintaining the highest standards of security and compliance.

## Join Our Team

We are a collaborative team of researchers, engineers, policy experts, business leaders, and operators dedicated to building the future of safe AI. If you are passionate about our mission and values, we encourage you to explore career opportunities at Anthropic. We offer competitive compensation, comprehensive benefits, and a supportive work environment.

## Contact Us

To learn more about Anthropic and our solutions, please visit our website or contact our sales team.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [120]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [121]:
def stream_brochure(company_name, url):
    # Prepare messages
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=get_brochure_user_prompt(company_name, url))
    ]
    
    response_text = ""
    display_handle = display(Markdown(""), display_id=True)

    # Stream tokens as they arrive
    for chunk in model.stream(messages):
        if chunk.content:
            response_text += chunk.content
            display_handle.update(Markdown(response_text))

In [122]:
stream_brochure("HuggingFace", "https://anthropic.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'}, {'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/jobs'}, {'type': 'products', 'url': 'https://www.anthropic.com/claude'}, {'type': 'products', 'url': 'https://www.anthropic.com/claude-code'}, {'type': 'products', 'url': 'https://www.anthropic.com/max'}, {'type': 'enterprise', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'customers', 'url': 'https://www.anthropic.com/customers'}, {'type': 'research', 'url': 'https://www.anthropic.com/research'}, {'type': 'news', 'url': 'https://www.anthropic.com/news'}, {'type': 'events', 'url': 'https://www.anthropic.com/events'}, {'type': 'solutions', 'url': 'https://www.anthropic.com/solutions/agents'}, {'type': 'solutions', 'url': 'https://www.anth

# Anthropic: AI for Humanity's Long-Term Well-Being

## Our Purpose

At Anthropic, we are dedicated to building reliable, interpretable, and steerable AI systems that serve humanity's long-term well-being. We focus on AI safety research, policy, and product design to demonstrate responsible AI development.

## Our Commitment

We believe AI will have a vast impact on the world and are committed to building systems people can rely on. We collaborate with civil society, government, academia, nonprofits, and industry to promote safety industry-wide.

## Our Values

*   **Act for the global good:** Maximize positive outcomes for humanity.
*   **Hold light and shade:** Understand and protect against potential risks while realizing beneficial outcomes.
*   **Be good to our users:** Cultivate generosity and kindness in all interactions.
*   **Ignite a race to the top on safety:** Inspire competition in developing the safest AI systems.
*   **Do the simple thing that works:** Take an empirical approach and prioritize impact over sophistication.
*   **Be helpful, honest, and harmless:** Communicate kindly and directly, assuming good intentions.
*   **Put the mission first:** Act swiftly together with a shared purpose.

## Our Products

### Claude

Claude is your thinking partner, connecting to your world and expanding what you can do, individually or with your team.

*   **Claude Opus:** Our most intelligent model, excelling in complex reasoning and coding.
*   **Claude Sonnet:** Balances intelligence and speed for everyday tasks.
*   **Claude Haiku:** Our fastest model, designed for quick responses.

#### Key Features:

*   **Connects to Your World:** Integrates documents, tools, and web knowledge.
*   **Solves Tough Challenges:** Handles complex questions and builds working code.
*   **Amplifies Creativity:** Turns ideas into polished projects.

#### Claude for Teams

*   Shorten the path from idea to impact with an AI assistant that taps into your team’s shared expertise.

### Claude Code

Unleash Claude's raw power directly in your terminal. Search million-line codebases instantly. Turn hours-long workflows into a single command. Your tools. Your workflow. Your codebase, evolving at thought speed.

### Claude Max

Combines Claude desktop & mobile apps and Claude Code in one subscription, with up to 20x more usage per session than Pro. That’s a lot of deep work potential.

## Solutions

We offer solutions for various industries, including:

*   **AI Agents:** Build powerful and collaborative AI agents.
*   **Code Modernization:** Modernize legacy codebases with Claude Code.
*   **Coding:** Write beautiful code and ship powerful products.
*   **Customer Support:** Build AI support agents with a human touch.
*   **Education:** Navigate AI in education with our responsible AI commitment.
*   **Financial Services:** Transform financial services with Claude.
*   **Government:** Deploy advanced AI capabilities with rigorous security controls.

## Research

We conduct frontier AI research to create safer, steerable, and more reliable models.

## Careers

Join our team and help us build the future of safe AI. We offer competitive benefits and a collaborative environment for researchers, engineers, policy experts, and operational leaders.

## Contact

Speak with our sales team or visit our website to learn more about how Anthropic can help you achieve your goals.

[Anthropic Website]


In [84]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'join', 'url': 'https://huggingface.co/join'}]}


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


```markdown
# Hugging Face: Building the Future of AI Together

**The AI community building the future.**

## Our Mission

To democratize good machine learning and foster collaboration within the AI community. We provide the platform where the machine learning community collaborates on models, datasets, and applications.

## The Hub: Your Central AI Platform

Hugging Face is the leading open-source platform for machine learning, offering a comprehensive ecosystem for AI development and deployment.

*   **Models:** Access over 1 million pre-trained models for various tasks, contributed by the community and industry leaders.
*   **Datasets:** Explore a vast library of datasets to train and evaluate your models.
*   **Spaces:** Build and share ML applications with ease, leveraging our powerful hosting infrastructure.

## Key Features & Benefits

*   **Collaboration:** Host and collaborate on unlimited public models, datasets, and applications.
*   **Open Source:** Built on a foundation of open-source tools, including Transformers, Diffusers, and Datasets.
*   **Accelerated Development:** Move faster with our open-source stack and optimized compute solutions.
*   **Versatile Modalities:** Support for text, image, video, audio, and 3D data.
*   **Portfolio Building:** Share your work, build your ML profile, and contribute to the community.

## Solutions

*   **Compute:** Deploy on optimized Inference Endpoints or update your Spaces applications to a GPU in a few clicks.
*   **Team & Enterprise Hub:** Give your team the most advanced platform to build AI with enterprise-grade security, access controls, and dedicated support.

## Enterprise-Grade Platform

Scale your organization with the Team & Enterprise Hub, offering:

*   **Single Sign-On (SSO):** Securely connect to your identity provider.
*   **Regions:** Select and manage the location of your repository data.
*   **Audit Logs:** Comprehensive logs for tracking actions and maintaining control.
*   **Resource Groups:** Granular access control for managing repositories.
*   **Token Management:** Centralized token control and custom approval policies.
*   **Analytics:** Track and analyze repository usage data.
*   **Advanced Compute Options:** Increased scalability and performance with ZeroGPU.
*   **Private Datasets Viewer:** Enable the Dataset Viewer on your private datasets.
*   **Priority Support:** Maximize platform usage with priority support.

## Pricing

We offer flexible pricing plans to suit your needs:

*   **Free:** The HF Hub is the central place to explore, experiment, collaborate and build technology with Machine Learning.
*   **Pro:** Boost your personal HF experience for $9 per month.
*   **Team:** Instant setup for growing teams at $20 per user per month.
*   **Enterprise:** Custom onboarding and enterprise features, starting at $50 per user per month.

## Join the Community

We are on a mission to democratize good machine learning. If that sounds like something you should be doing, why don't you join us!

*   [Website](https://huggingface.co)
*   [GitHub](https://github.com/huggingface)
*   [Twitter](https://twitter.com/huggingface)
*   [LinkedIn](https://www.linkedin.com/company/hugging-face/)
```

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>